In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
train = pd.read_csv('D:/Cours/MIASHS/PosterM2/train.csv')

# Passe ID en index
train.set_index('id', inplace=True)

# Affiche les 5 premières lignes avec toutes les colonnes
train.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
id,,,,,,,,,,,,,,,,,
0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [6]:
# Split des données à 80% pour l'entrainement "Tra" et 20% pour le test "Tes", la colonne NObeyesdad est la cible
from sklearn.model_selection import train_test_split
Tra, Tes = train_test_split(train, test_size=0.2, random_state=42)

In [9]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

#KNN

# Séparation des caractéristiques et de la cible
X_train = Tra.drop('NObeyesdad', axis=1)
y_train = Tra['NObeyesdad']
X_test = Tes.drop('NObeyesdad', axis=1)
y_test = Tes['NObeyesdad']

# Normalisation des données mais traiement des données catégorielles 
# avec LabelEncoder
label_encoder = LabelEncoder()
for col in X_train.select_dtypes(include='object'):
    X_train[col] = label_encoder.fit_transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])

# Normalisation des données numériques
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entraînement du modèle KNN
knn = KNeighborsClassifier(n_neighbors=5)  # Utiliser le nombre de voisins approprié
knn.fit(X_train_scaled, y_train)

# Test du modèle
predictions = knn.predict(X_test_scaled)
accuracy = accuracy_score(y_test, predictions)

print(f"L'exactitude du modèle est de : {accuracy * 100:.2f}%")


L'exactitude du modèle est de : 73.77%


In [10]:
# Mise en place d'une méthode grid search pour trouver le meilleur modèle
# On va jouer sur les hyperparamètres suivants : n_neighbors, weights, algorithm, metric

from sklearn.model_selection import GridSearchCV

# Création d'un dictionnaire des hyperparamètres
hyperparameters = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Création du modèle
knn = KNeighborsClassifier()

# Création de la méthode grid search
grid = GridSearchCV(knn, hyperparameters, cv=5)

# Entraînement du modèle
grid.fit(X_train_scaled, y_train)

# Affichage des meilleurs hyperparamètres
print(grid.best_params_)

# Test du modèle
predictions = grid.predict(X_test_scaled)
accuracy = accuracy_score(y_test, predictions)

print(f"L'exactitude du modèle est de : {accuracy * 100:.2f}%")

C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Pyth

{'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}
L'exactitude du modèle est de : 78.83%


In [11]:
# Réalisation d'une rédution de dimension avec LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=2)
X_train_lda = lda.fit_transform(X_train_scaled, y_train)
X_test_lda = lda.transform(X_test_scaled)

# Entraînement du modèle KNN
knn = KNeighborsClassifier(n_neighbors=5)  # Utiliser le nombre de voisins approprié
knn.fit(X_train_lda, y_train)

# Test du modèle
predictions = knn.predict(X_test_lda)
accuracy = accuracy_score(y_test, predictions)

print(f"L'exactitude du modèle est de : {accuracy * 100:.2f}%")

L'exactitude du modèle est de : 80.03%


In [12]:
# Mise en place d'une méthode grid search pour trouver le meilleur modèle
# On va jouer sur les hyperparamètres suivants : n_neighbors, weights, algorithm, metric
# avec la réduction de dimension

# Création du modèle
knn = KNeighborsClassifier()

# Création de la méthode grid search
grid = GridSearchCV(knn, hyperparameters, cv=5)

# Entraînement du modèle
grid.fit(X_train_lda, y_train)

# Affichage des meilleurs hyperparamètres
print(grid.best_params_)

# Test du modèle
predictions = grid.predict(X_test_lda)
accuracy = accuracy_score(y_test, predictions)

print(f"L'exactitude du modèle est de : {accuracy * 100:.2f}%")

C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Pyth

{'algorithm': 'auto', 'metric': 'euclidean', 'n_neighbors': 11, 'weights': 'uniform'}
L'exactitude du modèle est de : 81.53%


In [13]:
Kaggle = pd.read_csv('D:/Cours/MIASHS/PosterM2/test.csv')

# Passe ID en index
Kaggle.set_index('id', inplace=True)

Kaggle.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
id,,,,,,,,,,,,,,,,
20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


In [15]:
# Utilisation du meilleur modèle pour prédire les résultats de Kaggle et enregistrement dans un fichier CSV
X_Kaggle = Kaggle

# Normalisation des données mais traiement des données catégorielles
# avec LabelEncoder
label_encoder = LabelEncoder()
for col in X_Kaggle.select_dtypes(include='object'):
    X_Kaggle[col] = label_encoder.fit_transform(X_Kaggle[col])

# Normalisation des données numériques
scaler = StandardScaler()
X_Kaggle_scaled = scaler.fit_transform(X_Kaggle)

# Réduction de dimension avec LDA
X_Kaggle_lda = lda.transform(X_Kaggle_scaled)

# Prédiction des résultats
predictions = grid.predict(X_Kaggle_lda)

# Enregistrement des résultats dans un fichier CSV
submission = pd.DataFrame({'id': Kaggle.index, 'NObeyesdad': predictions})
submission.to_csv('D:/Cours/MIASHS/PosterM2/submission KNN.csv', index=False)

# Affichage des 5 premières lignes du fichier de soumission
submission.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_II
4,20762,Obesity_Type_III
